  # task_One
  ## Analyzing output from infoTracker
  _Date: 03/27/2017_

In [2]:
import pandas as pd

In [58]:
df = pd.read_csv('/home/savy/Documents/2017_master_internship/infoTracker/prot_est_Infos.csv')

In [127]:
df.head()

,Species name,Order name,Family name,#Protein,#EST
0,Catajapyx aquilonaris,Diplura,Japygidae,NaN,0
1,Orchesella cincta,Collembola,Entomobryidae,NaN,1146
2,Zootermopsis nevadensis,Blattodea,Termopsidae,14610.0,1
3,Nasutitermes exitiosus,Blattodea,Termitidae,NaN,0
4,Blattella germanica,Blattodea,Ectobiidae,NaN,8351


In [145]:
g = df.groupby(['Order name'])

In [158]:
maxEST = g['#EST'].max().to_dict()

In [186]:
hmmDict = {}
for index in df.index :
    for key in maxEST.keys():
        if df['Order name'][index] == key and df['#EST'][index] == maxEST[key]:
            hmmDict[df['Species name'][index]] = df['#EST'][index]

In [187]:
hmmDict

{'Acyrthosiphon pisum': 219173,
 'Amphinemura sulcicollis': 0,
 'Apis mellifera': 174597,
 'Baetis rhodani': 0,
 'Blattella germanica': 8351,
 'Bombyx mori': 575578,
 'Catajapyx aquilonaris': 0,
 'Dendroctonus ponderosae': 185437,
 'Drosophila melanogaster': 847942,
 'Ephemera danica': 0,
 'Frankliniella occidentalis': 13843,
 'Isoperla grammatica': 0,
 'Ladona fulva': 0,
 'Limnephilus lunatus': 0,
 'Locusta migratoria': 51188,
 'Mengenilla moldrzyki': 0,
 'Orchesella cincta': 1146,
 'Pediculus humanus corporis': 1188,
 'Timema cristinae': 0}

## Download fasta file from NCBI

### Test 1

In [2]:
from Bio import Entrez
Entrez.email = 'savandara.besse@gmail.com'

In [3]:
handle = Entrez.esearch(db="nucest", term='drosophila melanogaster')
record = Entrez.read(handle)['IdList']
handle.close()
record

['527482192', '392859066', '345421511', '342206684', '315905243', '315844150', '315823897', '315823706', '315820860', '315809918', '315803644', '315802413', '315798677', '311205018', '311147959', '302181891', '297251322', '297251281', '297251268', '297251209']

In [5]:
print("Parsing...")
record = SeqIO.read('test.fna', "fasta")
print(record)

Parsing...
ID: JZ478789.1
Name: JZ478789.1
Description: JZ478789.1 01RQ1.12 WSSP Duckweed cDNA library Landoltia punctata cDNA clone 01RQ1.12 5' similar to protein translocation protein, mRNA sequence
Number of features: 0
Seq('AGAGAGATCGTAGAGGGATTCGGTCAAGAGTACGTTCCCCAGAGGGTTCTACCG...GCA', SingleLetterAlphabet())
